In [134]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import covalent as ct

In [135]:
@ct.electron
def create_training_data(n):
    X = np.linspace(0, 2*np.pi, n)
    X.requires_grad = False
    Y = np.sin(X)
    return X, Y

In [136]:
@ct.electron
def create_test_data(n):
    X_test = np.linspace(0.2, 2*np.pi+0.2, n)
    Y_test = np.sin(X_test)
    return X_test, Y_test

In [137]:
@ct.electron
def plot_data(X_train, Y_train, X_test, Y_test):
    plt.scatter(X_train, Y_train, color='#219ebc', label='Training Data')
    plt.scatter(X_test, Y_test, color='#fb8500', alpha=0.5, label='Test Data')
    plt.legend()
    plt.show()

In [138]:
X, Y = create_test_data(5)
X_test, Y_test = create_training_data(5)
plot_data(X, Y, X_test, Y_test)

In [139]:
#@ct.electron
dev = qml.device('default.qubit', wires=1)
@qml.qnode(dev)
def quantum_circuit(datapoint, params):
    # Encode the input data as an RX rotation
    qml.RX(datapoint, wires=0)
    # # Create a rotation based on the angles in "params"
    qml.Rot(params[0], params[1], params[2], wires=0)
    # # We return the expected value of a measurement along the Z axis 
    return qml.expval(qml.PauliZ(wires=0))

In [140]:
@ct.electron
def loss_func(predictions):
    # This is a postprocessing step. Here we use a least squares metric 
    # based on the predictions of the quantum circuit and the outputs 
    # of the training data points.
    total_losses = 0
    for i in range(len(Y)):
        output = Y[i]
        prediction = predictions[i]
        loss = (prediction - output)**2
        total_losses += loss
    return total_losses

In [141]:
@ct.electron
def cost_fn(params): 
    # We get the predictions of the quantum circuit for a specific 
    # set of parameters along the entire input dataset
    predictions = [quantum_circuit(x, params)  for x in X]
    # We calculate the cost including any classical postprocessing
    cost = loss_func(predictions)
    return cost

In [142]:
@ct.electron
def gradient_descent():
    opt = qml.GradientDescentOptimizer(stepsize=0.3)
    params = np.array([0.1,0.1,0.1],requires_grad=True)
    for i in range (100):
        # Over each step the parameters change to give a better cost
        params, prev_cost = opt.step_and_cost(cost_fn,params) 
        if i%10 == 0:
             # We print the result after every 10 steps
            print(f'Step = {i} Cost = {cost_fn(params)}')
    return params

In [143]:
@ct.lattice
def fit_function():
    X, Y = create_test_data(6)
    X_test, Y_test = create_training_data(6)
    params = gradient_descent()
    return params

In [147]:
fit_function.draw()

In [ ]:
ct.dispatch(fit_function)

In [144]:
params = fit_function()
print(params)

Step = 0 Cost = 4.539661503138232
Step = 10 Cost = 0.021260772874236264
Step = 20 Cost = 0.0008706731235915955
Step = 30 Cost = 0.0003081475082736324
Step = 40 Cost = 0.0001801078236533777
Step = 50 Cost = 0.00011867294531622672
Step = 60 Cost = 8.410562985504932e-05
Step = 70 Cost = 6.27283423425901e-05
Step = 80 Cost = 4.858688146600381e-05
Step = 90 Cost = 3.874605002092442e-05
[-1.48145204  1.57053329  0.1       ]


In [145]:
test_predictions = []
for x_test in X_test:
    prediction = quantum_circuit(x_test,params)
    test_predictions.append(prediction)

In [146]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.plot(X, Y, c='#219ebc', label='Training data')
ax1.plot(X_test,Y_test, c='#fb8500', label='Test data')
ax1.scatter(X_test,test_predictions, s=30, c='k', marker="x", label='Model Predicitons')
plt.xlabel("Inputs")
plt.ylabel("Outputs")
plt.title("QML results")

plt.legend(loc='upper right');
plt.show()